In [1]:
import pandas as pd
import os
from openai import OpenAI

In [4]:
embeddings = pd.read_parquet("StandUp_Embeddings_Final.parquet")

print("Shape:", embeddings.shape)
print("Columns:", embeddings.columns.tolist())

print(embeddings.head)

embeddings_df = pd.DataFrame(embeddings)

Shape: (1175, 1543)
Columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '1

In [5]:
# Adding and defining Chroma lists

ids = embeddings_df["id"].astype(str).tolist()

titles = embeddings_df["title"].tolist()

contents = embeddings_df["content"].tolist()

vector_cols = [str(i) for i in range(1536)]

embeddings = embeddings_df[vector_cols].values.tolist()

metadatas = embeddings_df[["title", "section", "heading", "content"]].to_dict(orient= 'records')


In [6]:
# Using Chroma to build vector storage

import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection("StandUp")


In [7]:
collection.add(
    ids=ids,
    embeddings=embeddings,
    metadatas=metadatas,
    documents=contents
)

In [8]:

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def embed_query(text:str) -> list[float]:
    resp = client.embeddings.create(
        model = "text-embedding-ada-002",
        input = [text]
    )
    return resp.data[0].embedding

test_query = "My landlord was served an improvement notice, what does this mean and when does it start?"

query_vector = embed_query(test_query)



In [9]:
# The million dollar question! Will the program return the relevant laws and statutes


results = collection.query(
    query_embeddings=[query_vector], n_results = 10
)

# Match the id with title/section, since Chroma's metadata function did not work. The 'Title and Section' was always the first row of the dataset.

meta_lookup = {}

for _, row in embeddings_df.iterrows():
    statute_id = str(row["id"])
    meta_lookup[statute_id] = {
        "title": row["title"],
        "section": row["section"],
        "heading": row["heading"],
        "content": row["content"]
    }


matched_ids = results["ids"][0]
matched_docs = results["documents"][0]
distances = results["distances"][0]

for sid, docs, dist in zip(matched_ids, matched_docs, distances):
    data = meta_lookup[sid]
    print(f"Statute ID: {sid}\n Title and Section: {data["title"]} {data["section"]} \n Statute Heading: {data["heading"]} \n Statute Content: {docs} \n Distance: {dist}")



Statute ID: 613
 Title and Section: Housing Act 1988 s.14(3) 
 Statute Heading: Determination of rent by tribunal 
 Statute Content: For the purposes of subsection (2)(b) above, in relation to a notice which is referred by a tenant as mentioned in subsection (1) above, an improvement is a relevant improvement if either it was carried out during the tenancy to which the notice relates or the following conditions are satisfied, namelyâ (a) that it was carried out not more than twenty-one years before the date of service of the notice; and (b) that, at all times during the period beginning when the improvement was carried out and ending on the date of service of the notice, the dwelling-house has been let under an assured tenancy; and (c) that, on the coming to an end of an assured tenancy at any time during that period, the tenant (or, in the case of joint tenants, at least one of them) did not quit. 
 Distance: 0.26659783720970154
Statute ID: 730
 Title and Section: Housing Act 2004 s

In [10]:
# Creating an algorithm that logs the results into a csv for evaluation

query_id_list = []
statute_id_list = []
title_list = []
section_list = []
heading_list = []
content_list = []
distance_list = []

query_id = "6"

for sid, docs, dist in zip(matched_ids, matched_docs, distances):
    statute_id_str = str(sid)
    data = meta_lookup[statute_id_str]
    query_id_list.append(query_id)
    statute_id_list.append(statute_id_str)
    title_list.append(data.get("title", " "))
    section_list.append(data.get("section", " "))
    heading_list.append(data.get("heading", " "))
    content_list.append(docs)
    distance_list.append(dist)
    

In [11]:
# Creating a dataframe for all results
eval_df_6 = pd.DataFrame(list(zip(query_id_list, statute_id_list, title_list, section_list, heading_list, content_list, distance_list)),columns = ["query_id", "statute_id", "title_list", "section_list", "heading_list", "content_list", "distance_list"])



In [181]:
df_1 = pd.DataFrame(eval_df_1)


In [183]:
df_1.to_csv("query_final_1.csv")

In [64]:
df_2 = pd.DataFrame(eval_df_2)



In [65]:
df_2.to_csv("query_final_2.csv")

In [70]:
df_3 = pd.DataFrame(eval_df_3)


In [71]:
df_3.to_csv("query_final_3.csv")

In [76]:
df_4 = pd.DataFrame(eval_df_4)


In [77]:
df_4.to_csv("query_final_4.csv")

In [82]:
df_5 = pd.DataFrame(eval_df_5)

df_5.to_csv("query_final_5.csv")

In [12]:
df_6 = pd.DataFrame(eval_df_6)

df_6.to_csv("query_final_6.csv")


In [92]:
df_7 = pd.DataFrame(eval_df_7)

df_7.to_csv("query_final_7.csv")


In [97]:
df_8 = pd.DataFrame(eval_df_8)

df_8.to_csv("query_final_8.csv")


In [114]:
df_9 = pd.DataFrame(eval_df_9)

df_9.to_csv("query_final_9.csv")


In [119]:
df_10 = pd.DataFrame(eval_df_10)

df_10.to_csv("query_final_10.csv")


In [124]:
df_11 = pd.DataFrame(eval_df_11)

df_11.to_csv("query_final_11.csv")


In [129]:
df_12 = pd.DataFrame(eval_df_12)

df_12.to_csv("query_final_12.csv")


In [134]:
df_13 = pd.DataFrame(eval_df_13)

df_13.to_csv("query_final_13.csv")


In [139]:
df_14 = pd.DataFrame(eval_df_14)

df_14.to_csv("query_final_14.csv")


In [146]:
df_15 = pd.DataFrame(eval_df_15)

df_15.to_csv("query_final_15.csv")


In [151]:
df_16 = pd.DataFrame(eval_df_16)

df_16.to_csv("query_final_16.csv")


In [156]:
df_17 = pd.DataFrame(eval_df_17)

df_17.to_csv("query_final_17.csv")


In [163]:
df_18 = pd.DataFrame(eval_df_18)

df_18.to_csv("query_final_18.csv")


In [170]:
df_19 = pd.DataFrame(eval_df_19)

df_19.to_csv("query_final_19.csv")


In [176]:
df_20 = pd.DataFrame(eval_df_20)

df_20.to_csv("query_final_20.csv")